## Developing Applications with Apache Spark

Master scalable data processing with Apache Spark in this hands-on course. Learn to build efficient ETL pipelines, perform advanced analytics, and optimize distributed transformations using Spark’s DataFrame API. Explore grouping, aggregation, joins, set operations, and window functions. You'll also work with complex data types like arrays, maps, and structs, applying best practices for performance tuning.

---

### Prerequisites
You should meet the following prerequisites before starting this course:

- Basic programming knowledge
- Familiarity with Python
- Understanding of basic SQL (`SELECT`, `JOIN`, `GROUP BY`)
- Knowledge of data processing concepts
- Completion of **Introduction to Apache Spark** or prior Databricks experience

---


# Grouping and Aggregating Data

This demonstration will show how to perform grouping and aggregation operations using NYC Taxi trip data. We'll explore basic grouping, multiple aggregations, and window functions.

### Objectives
- Understand basic grouping operations in Spark
- Perform time-based analysis using aggregations
- Implement complex aggregations with multiple metrics
- Use window functions for advanced analytics
- Optimize aggregation performance

## Data Setup and Loading

First, let's load our taxi trip data and examine its structure.

In [0]:
from pyspark.sql.functions import *

# Read and displaying the taxi data
trips_df = spark.read.table("samples.nyctaxi.trips")

display(trips_df.limit(10))

## Basic Grouping Operations

Let's start with simple grouping operations to understand trip patterns by location.

In [0]:
# Count trips by pickup location, to show top 5 most popular pickup locations
location_counts = trips_df \
    .groupBy("pickup_zip") \
    .count() \
    .orderBy(desc("count"))

display(location_counts.limit(5))

## Combining Multiple Aggregations

Let's perform multiple aggregations by location using the `agg()` method

In [0]:
# Perform multiple aggregations by location, order by most popular pickup locations
location_stats = trips_df \
    .groupBy("pickup_zip") \
    .agg(
        count("*").alias("total_trips"),
        round(avg("trip_distance"), 2).alias("avg_distance"),
        round(avg("fare_amount"), 2).alias("avg_fare"),
        round(sum("fare_amount"), 2).alias("total_fare_amt")
    ) \
    .orderBy(desc("total_trips"))

display(location_stats.limit(5))

## Window Functions

Now let's use window functions for more advanced analytics.

In [0]:
from pyspark.sql.window import Window

# Create window specs for different ranking methods
window_by_trips = Window.orderBy(desc("total_trips"))
window_by_fare = Window.orderBy(desc("avg_fare"))

# Add different types of rankings
ranked_locations = location_stats \
    .withColumn("trips_rank", rank().over(window_by_trips)) \
    .withColumn("fare_rank", rank().over(window_by_fare)) \
    .withColumn("fare_quintile", ntile(5).over(window_by_fare))  # Divide into 5 groups by fare

In [0]:
ranked_locations.createOrReplaceTempView("ranked_locations")

In [0]:
%sql
select fare_quintile,min(avg_fare),max(avg_fare),count(*) as cnt_per_group from ranked_locations group by fare_quintile

In [0]:
# Displaying the results
display(ranked_locations.select(
    "pickup_zip", 
    "total_trips", 
    "avg_fare", 
    "avg_distance",
    "trips_rank",
    "fare_rank",
    "fare_quintile"
))

## Key Takeaways

1. **Basic Grouping**
   - Use `groupBy()` followed by aggregation method
   - Can group by multiple columns
   - Always check data distribution

2. **Window Functions**
   - Perfect for comparative analytics
   - Consider performance impact
   - Use appropriate window frame

3. **Best Practices**
   - Always alias aggregated columns
   - Handle null values appropriately
   - Consider data skew in grouping keys

# Lab - Grouping and Aggregating E-Commerce Data

In this lab, you'll practice working with grouping and aggregation in Spark using a dataset of e-commerce transactions. You'll perform various analyses to uncover patterns and insights in customer purchasing behavior.

### Objectives
- Use `groupBy` operations to summarize data
- Implement multiple aggregations
- Apply different ordering techniques
- (Bonus) Use window functions for advanced analytics

## Initial Setup

Load the retail transactions data and examine its structure.

In [0]:
from pyspark.sql.functions import *

## Read the e-commerce transactions data
transactions_df = spark.read.table("samples.bakehouse.sales_transactions")

## display a sample of the data
<FILL_IN>

## Basic Grouping Operations

Let's start with simple grouping operations to understand product sales patterns.

In [0]:
# 1. Group the data by products and count the number of sales
# 2. Order the results by the most popular products

# DataFrame Relational Operations in Spark

This demonstration shows how to effectively use joins and set operations with DataFrames, focusing on performance optimization and best practices.

### Objectives
- Understand different types of DataFrame joins
- Implement performance optimizations for joins
- Handle complex join scenarios
- Use set operations effectively
- Apply best practices for data skew

## Setup and Data Loading

First, let's load our sample retail data tables and examine their structures.

In [0]:
from pyspark.sql.functions import *

# Read the data 
transactions_df = spark.read.table("samples.bakehouse.sales_transactions")
customers_df = spark.read.table("samples.bakehouse.sales_customers")
franchises_df = spark.read.table("samples.bakehouse.sales_franchises")
suppliers_df = spark.read.table("samples.bakehouse.sales_suppliers")

In [0]:
# Examine schemas
transactions_df.printSchema()

In [0]:
customers_df.printSchema()

In [0]:
franchises_df.printSchema()

In [0]:
suppliers_df.printSchema()

## Basic Join Operations

Let's start with simple join operations to combine our data.

In [0]:
# Inner join example to enrich the transactions with store information
enriched_transactions = franchises_df.join(
    transactions_df,
    on="franchiseID",
    how="inner"
)

display(enriched_transactions)

In [0]:
# The "on" clause can contain an expression
enriched_transactions = franchises_df.join(
    transactions_df,
    on= transactions_df.franchiseID == franchises_df.franchiseID,
    how="inner"
)

display(enriched_transactions)

# This is particularly useful if the join key is named differently in both entities

In [0]:
# Please note how all fields from both dataframes are present in the result, a better practice is to project the columns you need from each entity
# We will also alias some of the columns to disambiguate column names
enriched_transactions = franchises_df \
    .select(
        "franchiseID", 
        col("name").alias("store_name"), 
        col("city").alias("store_city"), 
        col("country").alias("store_country")
        ) \
    .join(
        transactions_df,
        on="franchiseID",
        how="inner"
    )
    
display(enriched_transactions)

## Full Outer Join Operations

Let's analyze the relationships between dataframes and identify missing data using outer joins

In [0]:
# Let's analyze the relationship between franchises and suppliers using a full outer join
full_join = franchises_df \
    .withColumnRenamed("name", "franchise_name") \
    .join(
        suppliers_df.select("supplierID", col("name").alias("supplier_name")),
        on="supplierID",
        how="full_outer" # Doing outer join
    )

# Find records that would NOT appear in an inner join
# These are records where either franchises or suppliers data is null
non_matching_records = full_join.filter(
        col("franchiseID").isNull() | 
        col("supplier_name").isNull()
    ) \
    .select("franchiseID", "franchise_name", col("supplierID").alias("orphaned_supplier_id"))

display(non_matching_records)

### Using Spark SQL

Let's do this using Spark SQL now....


In [0]:
# Create temporary views
franchises_df.createOrReplaceTempView("franchises")
suppliers_df.createOrReplaceTempView("suppliers")

In [0]:
%sql
-- Let's do our outer join using SQL
SELECT 
    f.franchiseID,
    f.name as franchise_name,
    f.supplierID as orphaned_supplier_id
FROM franchises f
FULL OUTER JOIN suppliers s
ON f.supplierID = s.supplierID
WHERE f.franchiseID IS NULL OR s.name IS NULL

## Set Operations

Now let's explore relationships using set operations using the DataFrame API.

In [0]:
# Identify supplier IDs in each DataFrame
franchise_suppliers = franchises_df.select("supplierID").distinct()
all_suppliers = suppliers_df.select("supplierID").distinct()

# Find supplierIDs that are in franchises_df but not in suppliers_df
franchises_without_valid_suppliers = franchise_suppliers.subtract(all_suppliers)
display(franchises_without_valid_suppliers)

In [0]:
# Find the overlap - suppliers that exist in both tables
common_suppliers = franchise_suppliers.intersect(all_suppliers)
display(common_suppliers)

## Key Takeaways

1. **Join Strategy**
   - Use inner joins where keys exist in all dataframes
   - Use outer joins where there is a possibility that keys don't exist in both dataframes
   - Handle column name conflicts

2. **Performance Optimization**
   - Filter before joining
   - Project only needed columns
   - Handle skewed keys appropriately
   - Reference the smaller dataframe first; or
   - Use broadcast joins for small tables

In [0]:
## Group the data by products and count the number of sales
product_counts = <FILL-IN>

## Multiple Aggregations

Now let's perform multiple aggregations to get deeper insights.

In [0]:
# 1. Analyze sales by payment method
# 2. Calculate the total revenue, average transaction value, and count of transactions for each payment method
# 3. Order by total revenue (highest first)

In [0]:
## Analyze sales by payment method
payment_analysis = <FILL-IN>

## Bonus Challenge: Window Functions

If you have time, try using window functions for advanced analytics.

In [0]:

## First, calculate total revenue by product
product_revenue_df = transactions_df \
    .groupBy("product") \
    .agg(
        round(sum(col("totalPrice")), 2).alias("total_revenue")
    )

## Use window functions to add rankings
## Rank products by total revenue
<FILL-IN>

# Working with Complex Data Types in Spark

This demonstration shows how to effectively work with nested data structures in Spark, including structs, arrays, and maps, using real e-commerce data examples.

### Objectives
- Convert JSON string data to Spark SQL native complex types
- Understand and manipulate complex data types (Struct, Array, Map)
- Process nested JSON-like data structures
- Use the pivot and explode functions to reshape datasets as required

## Dataset Setup

Run the following cell to configure your working environment for this course. 


In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
import json

# Define the CSV-like data with JSON strings as a list of tuples
data = [
    (101, "Alice Smith", "true", "2023-01-15",
     """["hiking", "machine learning", "photography"]""",
     """[{"product_id": "P123", "name": "Laptop", "price": 1299.99, "date": "2023-03-10"}, {"product_id": "P456", "name": "External Monitor", "price": 249.99, "date": "2023-03-15"}]"""),
    
    (102, "Bob Johnson", "true", "2023-02-20",
     """["coding", "gaming", "reading"]""",
     """[{"product_id": "P789", "name": "Mechanical Keyboard", "price": 149.99, "date": "2023-04-05"}]"""),
    
    (103, "Charlie Williams", "false", "2022-11-05",
     """["golf", "cooking", "traveling"]""",
     """[]"""),
    
    (104, "Diana Garcia", "true", "2023-03-10",
     """["drawing", "yoga", "music"]""",
     """[{"product_id": "P234", "name": "Graphics Tablet", "price": 199.99, "date": "2023-03-25"}, {"product_id": "P567", "name": "Stylus Pen", "price": 49.99, "date": "2023-03-25"}, {"product_id": "P890", "name": "Design Software", "price": 299.99, "date": "2023-04-02"}]"""),
    
    (105, "Ethan Davis", "true", "2022-09-15",
     """["basketball", "programming", "movies"]""",
     """[{"product_id": "P321", "name": "Textbook", "price": 89.99, "date": "2023-01-10"}, {"product_id": "P654", "name": "Backpack", "price": 59.99, "date": "2023-01-10"}]"""),
    
    (106, "Fiona Miller", "true", "2023-04-01",
     """["social media", "writing", "photography"]""",
     """[{"product_id": "P987", "name": "Camera", "price": 599.99, "date": "2023-04-15"}]"""),
    
    (107, "George Wilson", "false", "2022-12-10",
     """["finance", "cycling", "chess"]""",
     """[{"product_id": "P111", "name": "Financial Software", "price": 199.99, "date": "2023-01-05"}, {"product_id": "P222", "name": "Wireless Mouse", "price": 29.99, "date": "2023-02-15"}]"""),
    
    (108, "Hannah Brown", "true", "2023-02-28",
     """["education", "reading", "gardening"]""",
     """[{"product_id": "P333", "name": "Educational Subscription", "price": 14.99, "date": "2023-03-01"}, {"product_id": "P444", "name": "Notebook Set", "price": 24.99, "date": "2023-03-01"}]"""),
    
    (109, "Ian Taylor", "true", "2022-10-20",
     """["cooking", "food", "travel"]""",
     """[{"product_id": "P555", "name": "Cooking Knives", "price": 179.99, "date": "2023-01-25"}, {"product_id": "P666", "name": "Recipe Book", "price": 39.99, "date": "2023-02-10"}, {"product_id": "P777", "name": "Spice Set", "price": 49.99, "date": "2023-03-20"}]"""),
    
    (110, "Julia Martinez", "true", "2023-03-15",
     """["law", "politics", "hiking"]""",
     """[{"product_id": "P888", "name": "Legal Reference Book", "price": 129.99, "date": "2023-04-10"}]""")
]

# Define the schema for the raw data
schema = StructType([
    StructField("user_id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("active", StringType(), True),
    StructField("signup_date", StringType(), True),
    StructField("interests", StringType(), True),
    StructField("recent_purchases", StringType(), True)
])

# Create DataFrame directly
df_raw = spark.createDataFrame(data, schema)

# Create Temp View
df_raw.createOrReplaceTempView("raw_user_data")

#### Querying the newly created table

In [0]:
%sql
select * from raw_user_data

## Convert from JSON Strings to StructTypes

Given raw data which includes nested JSON strings (arrays and/or objects), we will convert this data to native `StructTypes` in the DataFrame API.

#### Why Convert JSON Strings to StructTypes?

JSON strings in Spark DataFrames come with several inefficiencies:

1. **Parsing Overhead**: Every time you query JSON strings, Spark needs to parse them, adding computational overhead
2. **Memory Inefficiency**: JSON strings store field names repeatedly for every row, wasting memory
3. **No Type Safety**: JSON strings don't enforce data types, leading to potential errors
4. **Poor Query Performance**: Spark can't optimize queries on JSON string content as effectively
5. **Limited Predicate Pushdown**: Filter operations can't leverage columnar storage optimizations

### Steps to Convert JSON Strings to StructTypes

1. **Infer Schema**: Determine the structure of the JSON data (the `schema_of_json` function can be used for this)
2. **Apply Schema**: Use `from_json()` to convert strings to structured data
3. **Validate**: Ensure all data is correctly parsed and types are appropriate
4. **Optimize**: Once converted, optimize storage/processing if needed

### Benefits of StructTypes

1. **Columnar Storage**: Efficient storage with Parquet/Delta
2. **Type Safety**: Schema enforcement prevents data errors
3. **Query Optimization**: Spark can optimize queries better with typed data
4. **Predicate Pushdown**: Filters can be pushed down to storage layer
5. **Better Performance**: Faster queries and reduced memory usage

In [0]:
from pyspark.sql.functions import *

# Load some data which includes JSON strings
raw_user_data_df = spark.read.table("raw_user_data")

In [0]:
# Inspect the Data
raw_user_data_df.printSchema()

In [0]:
# Displaying the Dataframe 
display(raw_user_data_df)

1. View the data above and find columns **interests** and **recent_purchases**
2. **interests** is an array column of String elements and **recent_purchases** is column containing objects

In [0]:
# Interests is an array of strings, using predefined schema
interests_schema = ArrayType(StringType())

In [0]:
# Let's get the schema for the "recent_purchases" and cast all of the columns from the raw data set into a confirmed structure

# Take a sample of one value of the "recent_purchases" column, bring this back to the Driver
recent_purchases_json = raw_user_data_df.select("recent_purchases").limit(1).collect()[0][0]
print("Raw JSON string:", recent_purchases_json)


#### Use the **schema_of_json** Function to generate a Schema based upon a sample row of data

In many cases, especially with multiple nested complex structures, it is easiest to generate a schema based upon a sample of JSON data, we can do this using the schema_of_json Function.  

From the above dataset, we can see that **interests** is an array column of string elements, **recent_purchases** is an array column which contains objects.

In [0]:
# Get the schema for the recent_purchases JSON
recent_purchases_schema = schema_of_json(lit(recent_purchases_json))

In [0]:
# Parse columns with the correct schemas
parsed_users_df = raw_user_data_df.select(
    col("user_id").cast("integer"),
    col("name"),
    col("active").cast("boolean"),
    col("signup_date").cast("date"),
    from_json(col("interests"), interests_schema).alias("interests"),
    from_json(col("recent_purchases"), recent_purchases_schema).alias("recent_purchases")
)

# Examine the schema
parsed_users_df.printSchema()

In [0]:
# Now look at the data again, You will notice order has been changed for recent_purchases
display(parsed_users_df)

## Working with Arrays

Let's explore different ways to access and manipulate arrays.

In [0]:
# Use the array_size method to see the lengths of the array columns in the dataframe
display(
parsed_users_df.select(
    "user_id",
    array_size("interests").alias("number_of_interests"),
    array_size("recent_purchases").alias("number_of_recent_purchases")
    )
)

### The explode Method

The `explode` method is used to unnest array elements into records

In [0]:
# Let's start by simplifying the data by selecting only the columns we need
user_101s_interests_df = parsed_users_df.select("user_id", "interests").filter(parsed_users_df.user_id == 101)
display(user_101s_interests_df)

In [0]:
# Let's demonstrate explode, note how there are three rows associated with "user_id" 101 (one for each "interests" array element)
display(
    user_101s_interests_df.select(
        "user_id", 
        explode("interests").alias("interest")
    )    
)

### The collect_set and collect_list Methods

The `collect_set` and `collect_list` methods are aggregate functions (which typically operate on grouped data) to create arrays from column values.  

`collect_list` may include duplicate values, while `collect_set` removes duplicate array elements should they exist.

In [0]:
# Let's start by creating a new DataFrame with the "interests" column exploded
exploded_df = parsed_users_df.select("user_id", explode("interests").alias("interest"))
display(exploded_df)

In [0]:
# Let's use `collect_list` to collect all the "interests" values into a list for each "user_id"
user_interests_df = exploded_df.groupBy("user_id").agg(collect_list("interest").alias("interests"))
display(user_interests_df)

## Referencing Struct Fields

Let's explore how to access fields within a struct (an object with a predefined schema).

In [0]:
# First let's explode the "recent_purchases" column
exploded_purchases_df = parsed_users_df.select("user_id", explode("recent_purchases").alias("purchase"))
display(exploded_purchases_df)

In [0]:
# Use the dot notation to access struct fields
recent_purchases_df = exploded_purchases_df.select(
                        "user_id", 
                        col("purchase.date").alias("purchase_date"), 
                        col("purchase.product_id").alias("product_id"), 
                        col("purchase.name").alias("product_name"), 
                        col("purchase.price").alias("purchase_price")
                    )
display(recent_purchases_df)

In [0]:
# We can also use the getField() method to reference columns inside of structs, here's an example....

field_access_df = exploded_purchases_df.select(
    "user_id",
    col("purchase").getField("date").alias("purchase_date"),
    col("purchase").getField("product_id").alias("product_id"),
    col("purchase").getField("name").alias("product_name"),
    col("purchase").getField("price").alias("price")
)

display(field_access_df)

## Using the pivot Method

The `pivot` method in Spark allows you to transform row data into columnar format, creating a cross-tabulation. This is particularly useful for feature engineering when analyzing categorical data or when you need to reshape your data for reporting or visualization.

In [0]:
# Let's pivot the purchase data to show the count of each product purchased by each user
pivot_df = (recent_purchases_df
    .groupBy("user_id")
    .pivot("product_name")
    .agg(count("product_id").alias("quantity_purchased"))
)

# Display the result
display(pivot_df)

In [0]:
# Replace null values with zeros for better readability
pivot_df_no_nulls = (recent_purchases_df
    .groupBy("user_id")
    .pivot("product_name")
    .agg(count("product_id").alias("quantity_purchased"))
    .fillna(0)
)

# Display the result
display(pivot_df_no_nulls)

## Key Takeaways

1. **Struct Type Operations**:
   - Use dot notation for simple access
   - `getField()` for dynamic column access
   - Maintain schema clarity

2. **Array Operations**:
   - Use array functions for manipulation
   - Leverage explode for detailed analysis
   - Consider performance with large arrays

3. **Complex Aggregate Functions**:
   - Use the `collect_list` and `collect_set` methods to create arrays from grouped data
   - Use the `pivot` function to transform row values into columns for analysis and reporting


# Lab - Working with Complex Data Types in E-Commerce Data

In this lab, you'll practice working with complex data types in Spark, including handling JSON strings, converting them to structured types, and manipulating nested data structures.

## Scenario

You are a data engineer at an e-commerce company that collects data about customer orders, product reviews, and customer browsing behavior. The data contains nested structures that need to be properly processed for analysis.

### Objectives
- Convert JSON string data to Spark SQL native complex types
- Work with arrays and structs
- Use functions like explode, collect_list, and pivot
- Extract and analyze valuable insights from nested data

## Dataset Setup

Run the following cell to configure your working environment for this course. 

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
import json

# Define our sample e-commerce data with JSON strings
data = [
    (1001, "Jordan Smith", "jordan.smith@email.com", "2022-03-15",
     """["loyal", "premium", "tech-enthusiast"]""",
     """[
         {"order_id": "O8823", "date": "2023-01-05", "total": 799.99, "items": [
           {"product_id": "PHONE-256", "name": "Smartphone XS", "price": 699.99, "quantity": 1},
           {"product_id": "CASE-101", "name": "Phone Case", "price": 29.99, "quantity": 1},
           {"product_id": "CHGR-201", "name": "Fast Charger", "price": 49.99, "quantity": 1}
         ]},
         {"order_id": "O9012", "date": "2023-02-18", "total": 129.95, "items": [
           {"product_id": "HDPHN-110", "name": "Wireless Headphones", "price": 129.95, "quantity": 1}
         ]}
       ]""",
     """["smartphones", "accessories", "audio", "wearables"]"""
    ),
    
    (1002, "Alex Johnson", "alex.j@email.com", "2021-11-20",
     """["new", "standard", "home-office"]""",
     """[
         {"order_id": "O8901", "date": "2023-01-10", "total": 1299.99, "items": [
           {"product_id": "LAPTOP-15", "name": "Ultrabook Pro", "price": 1199.99, "quantity": 1},
           {"product_id": "MOUSE-202", "name": "Ergonomic Mouse", "price": 49.99, "quantity": 1},
           {"product_id": "KYBRD-303", "name": "Mechanical Keyboard", "price": 89.99, "quantity": 1}
         ]}
       ]""",
     """["laptops", "office-equipment", "monitors", "storage"]"""
    ),
    
    (1003, "Taylor Williams", "t.williams@email.com", "2022-08-05",
     """["standard", "gamer"]""",
     """[
         {"order_id": "O9188", "date": "2023-02-01", "total": 2099.97, "items": [
           {"product_id": "GPU-3080", "name": "Graphics Card RTX", "price": 899.99, "quantity": 1},
           {"product_id": "CPU-i9", "name": "Processor i9", "price": 499.99, "quantity": 1},
           {"product_id": "RAM-32GB", "name": "Gaming RAM 32GB", "price": 189.99, "quantity": 2},
           {"product_id": "MBOARD-Z", "name": "Gaming Motherboard", "price": 319.99, "quantity": 1}
         ]}
       ]""",
     """["gaming", "pc-components", "monitors", "accessories"]"""
    ),
    
    (1004, "Morgan Lee", "morgan.lee@email.com", "2022-06-10",
     """["standard", "photography"]""",
     """[
         {"order_id": "O9021", "date": "2023-01-15", "total": 3299.98, "items": [
           {"product_id": "CAM-DSLR", "name": "Professional Camera", "price": 2499.99, "quantity": 1},
           {"product_id": "LENS-50mm", "name": "Prime Lens", "price": 349.99, "quantity": 1},
           {"product_id": "TRIPOD-P", "name": "Premium Tripod", "price": 149.99, "quantity": 1},
           {"product_id": "SDCARD-128", "name": "Memory Card 128GB", "price": 79.99, "quantity": 3}
         ]},
         {"order_id": "O9254", "date": "2023-02-28", "total": 299.98, "items": [
           {"product_id": "BAG-CAM", "name": "Camera Bag", "price": 189.99, "quantity": 1},
           {"product_id": "CLEAN-KIT", "name": "Lens Cleaning Kit", "price": 29.99, "quantity": 1}
         ]}
       ]""",
     """["cameras", "photography", "lenses", "accessories"]"""
    ),
    
    (1005, "Casey Rivera", "casey.r@email.com", "2021-09-30",
     """["premium", "smart-home"]""",
     """[
         {"order_id": "O8765", "date": "2023-01-02", "total": 1029.95, "items": [
           {"product_id": "SMHUB-01", "name": "Smart Home Hub", "price": 249.99, "quantity": 1},
           {"product_id": "SMSPK-02", "name": "Smart Speaker", "price": 179.99, "quantity": 2},
           {"product_id": "SMBLB-03", "name": "Smart Bulbs Pack", "price": 119.99, "quantity": 3},
           {"product_id": "SMSENS-04", "name": "Motion Sensors", "price": 89.99, "quantity": 1}
         ]},
         {"order_id": "O9181", "date": "2023-02-15", "total": 349.98, "items": [
           {"product_id": "SMDLOCK-05", "name": "Smart Door Lock", "price": 249.99, "quantity": 1},
           {"product_id": "SMCAM-06", "name": "Indoor Camera", "price": 99.99, "quantity": 1}
         ]}
       ]""",
     """["smart-home", "security", "automation", "speakers"]"""
    )
]

# Define the schema for the raw data
schema = StructType([
    StructField("customer_id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("email", StringType(), True),
    StructField("registration_date", StringType(), True),
    StructField("tags", StringType(), True),
    StructField("recent_orders", StringType(), True),
    StructField("browsing_history", StringType(), True)
])

# Create DataFrame
ecommerce_df = spark.createDataFrame(data, schema)

# Create temporary view
ecommerce_df.createOrReplaceTempView("ecommerce_raw")

#### Querying the newly created table

In [0]:
%sql
select * from ecommerce_raw

## Load and Inspect Raw Data with JSON Strings

Load and examine the retail dataset which includes JSON strings.

In [0]:

## Read the sample dataset
events_df = spark.read.table("ecommerce_raw")

## Examine the schema and display sample data
<FILL-IN>

## Convert JSON Strings to Structured Types

The `tags`, `recent_orders`, and `browsing_history` columns contain JSON strings. Let's convert them to proper Spark structured types.

In [0]:
# 1. Get a sample of the JSON strings in each column
# 2. Infer schemas from the JSON samples
# 3. Convert the JSON strings to structured types using from_json and display the resulting DataFrame

In [0]:
## Get a sample of the JSON strings
<FILL-IN>

In [0]:
## Infer schemas from the JSON samples
<FILL-IN>

In [0]:
## Convert the JSON strings to structured types using from_json and display the resulting DataFrame
parsed_df = <FILL-IN>

## Working with Arrays

Now that we have proper structured data, let's analyze the customer tags and browsing history.

In [0]:
# 1. Calculate the number of tags and browsing history items for each customer
# 2. Explode the tags array to see all unique customer tags
# 3. Find the most common browsing categories across all customers
# HINT: use the `array_size` function or its alias `size`

In [0]:
## Calculate the number of tags and browsing history items for each customer
array_sizes_df = <FILL-IN>

In [0]:
## Explode tags to analyze customer categorization
exploded_tags_df = <FILL-IN>

In [0]:
## Find the most common customer tags
tag_counts_df = <FILL-IN>

In [0]:
# 1. Explode the recent_orders array to analyze individual orders
# 2. Calculate total revenue per customer

In [0]:
## Explode the recent_orders array to analyze individual orders
orders_df = <FILL-IN>

## Bonus Challenge: Analyze Customer Purchasing Patterns

Let's use the `collect_list` and `collect_set` aggregate functions to create summaries of customer purchasing patterns.

In [0]:
## First, create a flattened view of orders
order_items_df = orders_df.select(
    "customer_id",
    "name",
    "order.order_id",
    "order.date",
    explode("order.items").alias("item")
)

## Now extract the name field from each item
item_details_df = order_items_df.selectExpr(
    "customer_id",
    "name",
    "item.name as product_name"
)

# Inspect the data
display(item_details_df)

In [0]:
## Collect all products purchased by each customer, creating new columns called "all_products_purchased" and "unique_products_purchased" for each "customer_id"
customer_products_df = <FILL-IN>

# Optional - Basic ETL with the DataFrame API

This demonstration will walk through common ETL operations using the Flights dataset. We'll cover data loading, cleaning, transformation, and analysis using the DataFrame API.

### Objectives
- Implement common ETL operations using Spark DataFrames
- Handle data cleaning and type conversion
- Create derived features through transformations

NOTE: This section is optional and should be taught at start to give intro to Dataframe API

## Data Loading and Inspection

First, let's load and inspect the flight data.

In [0]:
# Read the flights data
flights_df = spark.read.table("databricks_airline_performance_data.v01.flights_small")

In [0]:
# Print the schema
flights_df.printSchema()

In [0]:
# Visually inspect a subset of the data
display(flights_df.limit(10))

In [0]:
# Let's remove columns we dont need, remember "filter early, filter often"
flights_required_cols_df = flights_df.select(
    "Year",
    "Month",
    "DayofMonth",
    "DepTime",
    "FlightNum",
    "ActualElapsedTime",
    "CRSElapsedTime",
    "ArrDelay")

# Alternatively we could have used the drop() method to remove the columns we didnt want...

In [0]:
# Get a count of the source data records
initial_count = flights_required_cols_df.count()

print(f"Source data has {initial_count} records")

In [0]:
# Let's examine the data for invalid values, these can include nulls or invalid values for string columns "ArrDelay", "ActualElapsedTime", "DepTime" which we intend on performing mathematical opeations on, we can use the Spark SQL COUNT_IF function to perform the analysis

# Register the DataFrame as a temporary SQL table with cast columns
flights_required_cols_df \
    .selectExpr(
        "Year",
        "Month",
        "DayofMonth",
        "TRY_CAST(DepTime AS INT) AS DepTime",
        "FlightNum",
        "TRY_CAST(ActualElapsedTime AS INT) AS ActualElapsedTime",
        "CRSElapsedTime",
        "TRY_CAST(ArrDelay AS INT) AS ArrDelay"
    ) \
    .createOrReplaceTempView("flights_temp")

## Data Cleaning

The flights data contains some invalid and missing values, lets find them and clean them (in this case we will drop them)

In [0]:
# To drop rows where any specified columns are null, we can use the na.drop DataFrame method
non_null_flights_df = flights_required_cols_df.na.drop(
    how='any',
    subset=['CRSElapsedTime']
)

In [0]:
from pyspark.sql.functions import col

# Let's remove rows with invalid values for "ArrDelay", "ActualElapsedTime" and "DepTime" columns
flights_with_valid_data_df = non_null_flights_df.filter(
    col("ArrDelay").try_cast("integer").isNotNull() & 
    col("ActualElapsedTime").try_cast("integer").isNotNull() &
    col("DepTime").try_cast("integer").isNotNull()
)

In [0]:
# Now that we know "ArrDelay" and "ActualElapsedTime" contain integer values only, lets cast them from strings to integers (replacing the existing columns)
clean_flights_df = flights_with_valid_data_df \
    .withColumn("ArrDelay", col("ArrDelay").try_cast("integer")) \
    .withColumn("ActualElapsedTime", col("ActualElapsedTime").try_cast("integer"))

clean_flights_df.printSchema()

## Data Enrichment

Now let's create a useful derived column to categorize delays.

In [0]:
# Let's start by deriving the "FlightDateTime" column from the "Year", "Month", "DayofMonth", "DepTime" columns, then drop the constituent columns
from pyspark.sql.functions import col, make_timestamp_ntz, lpad, substr, lit, pmod

flights_with_datetime_df = clean_flights_df.withColumn(
    "FlightDateTime",
    make_timestamp_ntz(
        col("Year"),
        col("Month"),
        col("DayofMonth"),
        pmod(substr(lpad(col("DepTime"), 4, "0"), lit(1), lit(2)).try_cast("integer"), lit(24)),
        substr(lpad(col("DepTime"), 4, "0"), lit(3), lit(2)).try_cast("integer"),
        lit(0)
    )
).drop("Year", "Month", "DayofMonth", "DepTime")

# Show the result
display(flights_with_datetime_df.limit(10))

In [0]:
# OK now lets derive the "ElapsedTimeDiff" column from the "ActualElapsedTime" and "CRSElapsedTime" columns

from pyspark.sql.functions import col

flights_with_elapsed_time_diff_df = flights_with_datetime_df.withColumn(
    "ElapsedTimeDiff", col("ActualElapsedTime") - col("CRSElapsedTime")
    ).drop("ActualElapsedTime", "CRSElapsedTime")

display(flights_with_elapsed_time_diff_df.limit(10))

In [0]:
# Now lets categorize the "ArrDelay" column into categories: "On Time", "Slight Delay", "Moderate Delay", "Severe Delay"

from pyspark.sql.functions import when

enriched_flights_df = flights_with_elapsed_time_diff_df \
    .withColumn("delay_category", when(col("ArrDelay") <= 0, "On Time")
        .when(col("ArrDelay") <= 15, "Slight Delay")
        .when(col("ArrDelay") <= 60, "Moderate Delay")
        .otherwise("Severe Delay")) \
       .drop("ArrDelay")

In [0]:
# Displaying the result 
display(enriched_flights_df)